In [ ]:
!nvidia-smi

In [ ]:
!pip install textblob 'keras-nlp' 'keras-preprocessing' 'tensorflow-text==2.15.0' np_utils swifter

In [ ]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda, ELU, Conv1D, MaxPooling1D, Dropout, LSTM, TimeDistributed, SpatialDropout1D, Bidirectional
from keras import Model, Input
from keras.callbacks import EarlyStopping
import np_utils
import swifter
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from tensorflow.keras.layers.experimental import preprocessing
import keras_nlp
import os
import time
import sys
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk
import pickle
from tensorflow.nn import leaky_relu

import re
import warnings
from sklearn.model_selection import train_test_split
from textblob import TextBlob
from collections import defaultdict


TRACE = False
embedding_dim = 100
rnn_units = 128
epochs=100
buffer_size = 256
max_len = 50
# Batch size
batch_size = 256
min_count_words = 3
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

In [ ]:
%%writefile get_data.sh
if [ ! -f ner_dataset.csv ]; then
  wget -O ner_dataset.csv https://www.dropbox.com/s/mbfv0x988mdj89h/ner_dataset.csv?dl=0
fi


In [ ]:
!bash get_data.sh

In [ ]:
data= pd.read_csv("./ner_dataset.csv",encoding="latin1")
data = data.fillna(method='ffill')
data.head()

In [ ]:
print("Unique Words in corpus:",data['Word'].nunique())
print("Unique Tag in corpus:",data['Tag'].nunique())

In [ ]:
words = list(set(data['Word'].values))
words.append("ENDPAD")
num_words = len(words)
tags = list(set(data['Tag'].values))
num_tags = len(tags)

In [ ]:
class SentenceGetter(object):
  def __init__(self,data):
    self.n_sent = 1 #counter
    self.data = data
    agg_func = lambda s:[(w,p,t) for w,p,t in zip(s['Word'].tolist(),s['POS'].tolist(),s['Tag'].tolist())]
    self.grouped = self.data.groupby("Sentence #").apply(agg_func)
    self.sentences = [s for s in self.grouped]



getter = SentenceGetter(data)
sentences = getter.sentences

In [ ]:
sentences[0]

In [ ]:
word2idx =  {w : i+1 for i,w in enumerate(words)}
tag2idx  =  {t : i for i,t in enumerate(tags)}

In [ ]:
X = [[word2idx[w[0]] for w in s]for s in sentences]
X = pad_sequences(maxlen = max_len , sequences =X, padding='post', value =num_words-1)

y = [[tag2idx[w[2]] for w in s]for s in sentences]
y = pad_sequences(maxlen = max_len , sequences =y, padding='post', value =tag2idx["O"])
y = [to_categorical(i, num_classes=num_tags) for i in y]
x_train,x_test,y_train,y_test = train_test_split(X,y, test_size=0.1, random_state=1)


In [ ]:
model = Sequential()
model.add(Input(shape = (max_len,)))
model.add(Embedding(input_dim=num_words,output_dim=max_len,input_length=max_len))
model.add(SpatialDropout1D(0.1))
model.add(Bidirectional(LSTM(units=100, activation = 'tanh', recurrent_activation = 'sigmoid', return_sequences=True)))
model.add(TimeDistributed(Dense(num_tags,activation='softmax')))

In [ ]:
model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=[tf.keras.metrics.KLDivergence(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.summary()

In [ ]:
early_stopping = EarlyStopping(monitor='kullback_leibler_divergence',patience=5, verbose=0, mode='min',restore_best_weights=False)
history = model.fit(x_train, np.array(y_train), validation_split = 0.2, batch_size = batch_size, epochs = epochs, verbose = 1, callbacks=[early_stopping])

In [ ]:
model.evaluate(x_test, np.array(y_test))

In [ ]:
i = np.random.randint(0, x_test.shape[0])
p = model.predict(np.array([x_test[i]]))
p = np.argmax(p, axis=-1)

y_true = np.argmax(np.array(y_test), axis=-1)[i]
print("{:15}{:5}\t {}\n".format("Word","True","Pred"))
print("-"*30)
for w,true,pred in zip(x_test[i], y_true, p[0]):
  print("{:15}{:5}\t{}".format(words[w-1], tags[true],tags[pred]))
